In [1]:
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image

from network.styler import Unet
from loss.loss import CLIPLoss
from utils.func import get_features,vgg_normalize

import time

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

lr1 = 0.001
lr2 = 0.0003
# lr_fast = 0.0002
# lr_slow = 0.00005

# model = Unet(device).to(device)
model = Unet().to(device)
cliploss = CLIPLoss(device)
mseloss = torch.nn.MSELoss()
vgg = torchvision.models.vgg19(pretrained=True).features.to(device)
for x in vgg.parameters():
    x.requires_grad = False

topil = transforms.ToPILImage()
topic = transforms.ToTensor()

dir_lambda = 500
content_lambda = 150
patch_lambda = 9000
norm_lambda = 0.002
gol_lambda = 300

In [3]:
file_u = open("neo.txt", "r")
loss_li = file_u.readline()
# loss_li = [float(x) for x in loss_li.split()]
loss_li = None
if not loss_li:
    loss_li = [0]*100

cur_times = int(file_u.readline())


In [4]:
def train(iteration1, iteration2, pic, source, target):
    input = pic

    # pic_f = get_features(vgg_normalize(pic), vgg)
    # print(model.parameters())
    opt = optim.Adam(model.parameters(), lr=lr1)
    for i in range(iteration1):
        opt.zero_grad()
        neo_pic = model(input)
        loss = mseloss(pic, neo_pic) * 1

        # loss = 0
        # neo_pic_f = get_features(vgg_normalize(neo_pic), vgg)
        # loss += torch.mean((pic_f['conv4_2'] - neo_pic_f['conv4_2']) ** 2)
        # loss += torch.mean((pic_f['conv5_2'] - neo_pic_f['conv5_2']) ** 2)

        loss.backward()
        opt.step()

        print("iter:", i + 1, "loss:", loss.item())

        # pil = topil(neo_pic.squeeze(0).cpu())
        # if ((i + 1) % 50) == 0:
        #     pil.save(f"./pic1/{(i + 1) // 50}.jpg")
    # neo_pic = model(input)
    # pil = topil(neo_pic.squeeze(0).cpu())
    # # pil.save(f"{source}-{target}.jpg")
    # pil.save(path)


    # torch.save(model,'unet.pth')

    # model = torch.load('unet.pth')

    pic_f = get_features(vgg_normalize(pic),vgg)

    opt = optim.Adam(model.parameters(), lr=lr2)
    # opt_fast = optim.Adam(model.parameters(), lr=lr2)
    # opt_slow = optim.Adam(model.parameters(), lr=lr_fast)
    # opt_loss = optim.Adam(cliploss.parameters(), lr=lr_slow)
    for i in range(iteration2):


        opt.zero_grad()
        # opt_slow.zero_grad()
        # opt_fast.zero_grad()

        neo_pic = model(input)

        dir_loss = 0
        dir_loss += cliploss.forward_dir(pic, source, neo_pic, target)

        gol_loss = 0
        # gol_loss += cliploss.forward_gol(pic, source, neo_pic, target)

        content_loss = 0
        # content_loss += mseloss(pic, neo_pic)
        neo_pic_f = get_features(vgg_normalize(neo_pic), vgg)
        content_loss += torch.mean((pic_f['conv4_2'] - neo_pic_f['conv4_2']) ** 2)
        content_loss += torch.mean((pic_f['conv5_2'] - neo_pic_f['conv5_2']) ** 2)

        patch_loss = 0
        patch_loss += cliploss.forward_patch(pic, source, neo_pic, target)

        norm_loss = 0
        norm_loss += cliploss.forward_prior(pic, source, neo_pic, target)

        loss = dir_loss * dir_lambda + \
               content_loss * content_lambda + \
               patch_loss * patch_lambda + \
               norm_loss * norm_lambda + \
               gol_loss * gol_lambda



        # patch_loss_fast,patch_loss_slow = cliploss.forward_patch_sec(pic, source, neo_pic, target)
        # patch_loss_fast.backward(retain_graph=True)
        # patch_loss_slow.backward(retain_graph=True)


        loss.backward()
        opt.step()

        # opt_fast.step()
        # opt_slow.step()

        loss_li[i]+=(loss.item())

        print("iter:", i + 1, "loss:", loss.item())

        with torch.no_grad():
            neo_pic = model(input)
            pil = topil(neo_pic.squeeze(0).cpu())
            pil = transforms.Resize(size=ori_size, interpolation=Image.BICUBIC)(pil)
            pil.save(f"mid.jpg")


        # pil = topil(neo_pic.squeeze(0).cpu())
        # if ((i + 1) % 10) == 0:
        #     pil.save(f"./pic2/{(i + 1) // 10}.jpg")

    # return  model(input)
    # neo_pic = model(input)
    # pil = topil(neo_pic.squeeze(0).cpu())
    # # pil.save(f"{source}-{target}.jpg")
    # pil.save(path)


In [5]:
pil = Image.open(f"./source_pic/emma man.jpg")
ori_size = pil.size[::-1]
pil = transforms.Resize(size=(512, 512), interpolation=Image.BICUBIC)(pil)
pic = topic(pil).unsqueeze(0).to(device)
# pic = torch.ones(1, 3, 512, 512).to(device)
pic.requires_grad = False

source = "photo"
# source = "white shirt"
# source = "man"
# source = "anime"

# target = "Neon light"
# target = "metal"
# target = "Van gogh"
# target = "Japanese anime"
# target = "woman"
target = "snowy"
# target = "white wool"
# target = "black suit"
# target = "sketch"
# target = "pop art of night city"
# target = "starry night by Van Gogh"
# target = "the scream by Edvard Munch"
# target = "Chinese Ink and wash painting"
# target = "The great wave off kanagawa by Hokusai"

path = "result/result12.jpg"

C:\Users\win10\AppData\Local\Temp\ipykernel_16440\683721222.py:3: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  pil = transforms.Resize(size=(512, 512), interpolation=Image.BICUBIC)(pil)
E:\Anaconda\envs\sth\lib\site-packages\torchvision\transforms\transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [6]:
start = time.time()
train(100, 100, pic, source, target)
end = time.time()
usetime = end - start
print(f"usetime: {usetime}")

iter: 1 loss: 0.06487759947776794
iter: 2 loss: 0.06040358543395996
iter: 3 loss: 0.03927142173051834
iter: 4 loss: 0.027946343645453453
iter: 5 loss: 0.020156491547822952
iter: 6 loss: 0.013150975108146667
iter: 7 loss: 0.008388716727495193
iter: 8 loss: 0.005981182213872671
iter: 9 loss: 0.004255324602127075
iter: 10 loss: 0.003868627827614546
iter: 11 loss: 0.0034963577054440975
iter: 12 loss: 0.0032865942921489477
iter: 13 loss: 0.0030545229092240334
iter: 14 loss: 0.002709121908992529
iter: 15 loss: 0.0024101953022181988
iter: 16 loss: 0.002179228700697422
iter: 17 loss: 0.001995484810322523
iter: 18 loss: 0.0018243843223899603
iter: 19 loss: 0.0016827614745125175
iter: 20 loss: 0.0015952258836477995
iter: 21 loss: 0.001503292005509138
iter: 22 loss: 0.0013940322678536177
iter: 23 loss: 0.0013015121221542358
iter: 24 loss: 0.0012250059517100453
iter: 25 loss: 0.0011668835068121552
iter: 26 loss: 0.0011299209436401725
iter: 27 loss: 0.001105481176637113
iter: 28 loss: 0.00108203152

C:\Users\win10\AppData\Local\Temp\ipykernel_16440\3951932543.py:94: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  pil = transforms.Resize(size=ori_size, interpolation=Image.BICUBIC)(pil)


KeyboardInterrupt: 

In [ ]:
neo_pic = model(pic)
pil = topil(neo_pic.squeeze(0).cpu())
pil = transforms.Resize(size=ori_size, interpolation=Image.BICUBIC)(pil)
pil.save(path)



# 186.4968602657318


In [ ]:
# with open(file = "neo.txt", mode = "w") as file:
#     for i in loss_li:
#         file.write(str(i)+" ")
#     file.write("\n")
#     file.write(str(cur_times+1))


In [ ]:
import matplotlib.pyplot as plt

x = [i for i in range(100)]
loss_li = [x/(cur_times+1) for x in loss_li]
plt.plot(x,loss_li,color="red",marker="o")


